In [1]:
import pandas as pd
import numpy as np
import requests as rq
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from lxml import etree
import time

In [2]:
def get_links():
    links = []

    h2_tags = sp.find_all('h2')
    for h2_tag in h2_tags:
        a_tag = h2_tag.find('a')
        if a_tag and 'href' in a_tag.attrs:
            links.append(a_tag['href'])
    return links

def get_article_title():
    try:
        title = sp.find('h1', class_='article-title').text
        return title if title else 0
    except:
        return 0

def get_article_date():
    try:
        date = sp.find('time').text[:10]
        return date
    except:
        return 0

def get_article_summary():
    try:
        div_tag = sp.find_all('div', class_="articleimg")
        for text in div_tag:
            summary = text.get_text()
        return summary 
    except:
        return 0

def get_article_text():
    try:
        text = ''

        div_tags = sp.find_all('div', class_="post-article text-align-left")
        for div_tag in div_tags:
            p_tags = div_tag.find_all('p')
            for p_tag in p_tags:
                text += p_tag.get_text()    
        return text
    except:
        return 0

In [3]:
column_name = ['報導標題', '報導日期', '報導大意', '報導內文']
df = pd.DataFrame([[0, 0, 0, 0]], columns=column_name)

In [4]:
report_type = 'hotTopic/41'

for page_num in range(1, 11):
    print(f'Page {page_num} start')
    url = 'https://news.pts.org.tw/' + report_type + '?page=' + str(page_num)
    html = rq.get(url)
    sp = soup(html.text, 'lxml')
    links_in_this_page = get_links()
    for link in links_in_this_page:
        html = rq.get(link)
        sp = soup(html.text, 'lxml')
        title = get_article_title()
        date = get_article_date()
        summary = get_article_summary()
        text = get_article_text()
        df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
    print(f'Page {page_num} end')
    
df = df.drop(0)
df = df.reset_index(drop=True)

Page 1 start


C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame

Page 1 end
Page 2 start


C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame

Page 2 end
Page 3 start


...

C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame

Page 9 end
Page 10 start


C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame

Page 10 end


C:\Users\user\AppData\Local\Temp\ipykernel_21556\2919665128.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame([[title, date, summary, text]], columns=column_name), ignore_index=True)


In [5]:
df

,報導標題,報導日期,報導大意,報導內文
0,台灣選手東奧創佳績 背後團隊期待選手培育系統有全面性發展,2021-09-02,台灣選手在本屆奧運中表現傑出，獲得2金、4銀、6銅成績，刷新史上最佳紀錄。選手們在回台後出席...,代表台灣，征戰東京奧運的幾位選手依序上台，從台北市長柯文哲手中接過獎狀或獎盃。台北市表揚運動...
1,輔大「婞淳金牌廳」今舉行揭牌 郭婞淳：會在2024年巴黎奧運持續為國爭光,2021-09-02,舉重女將郭婞淳在東京奧運中拿下金牌，她任教的輔仁大學，特別規劃建置「輔仁大學運動名人堂—婞淳...,舉重女將郭婞淳和輔大的師長們一起揭牌，笑得格外的開心。郭婞淳在東奧賽事中，為我國拿下本屆首面...
2,總統府明辦台灣英雄派對 感謝東奧國手,2021-08-31,台灣選手在東京奧運獲得2金4銀6銅，創下史上最佳成績，因為疫情，因此歡慶英雄谷的遊行延後，但...,俏皮五連拍，訪問時互相舖梗，東奧舉重金牌郭婞淳、柔道銀牌楊勇緯，拳擊第五名陳念琴，返國後首次...
3,楊勇緯、林真豪回高中母校 將奪獎戰袍留下勉勵學弟妹,2021-08-13,東京奧運柔道銀牌選手楊勇緯，以及女子柔道打進前八強的林真豪，回台中的第一個公開活動就是回到台...,才回到台中的母校新民高中，奪下東京奧運柔道銀牌的楊勇緯，以及打進女子柔道前八強的林真豪，受到...
4,東奧最後一批選手今搭機回國 田徑總教練再為經濟艙爭議緩頰,2021-08-09,我國選手在東京奧運，創下史上最佳成績二金、四銀、六銅。東奧昨晚閉幕後，包括田徑選手鄭兆村等最...,一起合照，比出「讚」的手勢，9日返國的陳傑、鄭兆村等田徑好手，以及代表團人員，共37人一起感...
...,...,...,...,...
136,東奧聖火造型出爐 櫻花金火焰象徵團結,2019-03-21,0,
137,日本奧委會主席請辭 預計六月底卸任,2019-03-20,0,
138,日本東奧招募8萬志工 逾20萬人應徵,2019-03-12,0,
139,國際奧會主席：歡迎「中華台北」 參加東奧,2018-11-29,0,


In [ ]:
report_type = 'test'
df.to_excel(f"data/pts_{report_type}.xlsx", encoding='utf-8-sig', index=False)
df.to_csv(f"data/pts_{report_type}.csv", encoding='utf-8-sig', index=False)

In [6]:
df

,報導標題,報導日期,報導大意,報導內文
0,台灣選手東奧創佳績 背後團隊期待選手培育系統有全面性發展,2021-09-02,台灣選手在本屆奧運中表現傑出，獲得2金、4銀、6銅成績，刷新史上最佳紀錄。選手們在回台後出席...,代表台灣，征戰東京奧運的幾位選手依序上台，從台北市長柯文哲手中接過獎狀或獎盃。台北市表揚運動...
1,輔大「婞淳金牌廳」今舉行揭牌 郭婞淳：會在2024年巴黎奧運持續為國爭光,2021-09-02,舉重女將郭婞淳在東京奧運中拿下金牌，她任教的輔仁大學，特別規劃建置「輔仁大學運動名人堂—婞淳...,舉重女將郭婞淳和輔大的師長們一起揭牌，笑得格外的開心。郭婞淳在東奧賽事中，為我國拿下本屆首面...
2,總統府明辦台灣英雄派對 感謝東奧國手,2021-08-31,台灣選手在東京奧運獲得2金4銀6銅，創下史上最佳成績，因為疫情，因此歡慶英雄谷的遊行延後，但...,俏皮五連拍，訪問時互相舖梗，東奧舉重金牌郭婞淳、柔道銀牌楊勇緯，拳擊第五名陳念琴，返國後首次...
3,楊勇緯、林真豪回高中母校 將奪獎戰袍留下勉勵學弟妹,2021-08-13,東京奧運柔道銀牌選手楊勇緯，以及女子柔道打進前八強的林真豪，回台中的第一個公開活動就是回到台...,才回到台中的母校新民高中，奪下東京奧運柔道銀牌的楊勇緯，以及打進女子柔道前八強的林真豪，受到...
4,東奧最後一批選手今搭機回國 田徑總教練再為經濟艙爭議緩頰,2021-08-09,我國選手在東京奧運，創下史上最佳成績二金、四銀、六銅。東奧昨晚閉幕後，包括田徑選手鄭兆村等最...,一起合照，比出「讚」的手勢，9日返國的陳傑、鄭兆村等田徑好手，以及代表團人員，共37人一起感...
...,...,...,...,...
136,東奧聖火造型出爐 櫻花金火焰象徵團結,2019-03-21,0,
137,日本奧委會主席請辭 預計六月底卸任,2019-03-20,0,
138,日本東奧招募8萬志工 逾20萬人應徵,2019-03-12,0,
139,國際奧會主席：歡迎「中華台北」 參加東奧,2018-11-29,0,
